In [1]:
import pandas as pd
import numpy as np

In [2]:
def map_age(age):
    if age < 60: return '< 60'
    elif age >= 60 and age < 70: return '60-69'
    elif age >= 70 and age < 80: return '70-69'
    elif age >= 80: return '> 80'
    

def map_age2(age):
    if age < 60: return '< 60'
    elif age >= 60: return '> 60'
    
def map_sexo(sexo):
    
    if sexo == 'Masc': return 'M'
    elif sexo == 'Fem': return 'F'
    elif sexo == 'Mas': return 'M'
    elif sexo == 'FEM': return 'F'
    else: return 'Não informado'

In [3]:
data_obitos = pd.read_csv("data_general/obitos_sergipe.csv")

In [4]:
data_obitos.rename(columns={
    'IDADE': 'idade', 
    'SEXO': 'sexo', 
    'MUNICÍPIO': 'municipio', 
    'COMORBIDADE': 'comorbidade', 
    'DATA ÓBITO': 'data_obito',
    'DATA CONFIRMAÇÃO': 'data_confirmacao',
},
                  inplace=True)

data_obitos['idade_faixa'] = data_obitos['idade'].apply(map_age)        
data_obitos['sexo'] = data_obitos['sexo'].apply(map_sexo)
data_obitos

,idade,sexo,municipio,comorbidade,data_obito,data_confirmacao,idade_faixa
0,52,M,Itabaiana,Sem Comorbidade,14/08/2021,26/06/2021,< 60
1,75,M,Barra dos Coqueiros,Sem Comorbidade,14/08/2021,06/08/2021,70-69
2,71,M,Aracaju,Sem Comorbidade,13/08/2021,10/08/2021,70-69
3,53,M,Aracaju,"Hipertensão Arterial Sistemica, Obesidade",09/08/2021,15/04/2021,< 60
4,59,M,Aracaju,"Hipertensão Arterial Sistemica, Obesidade, Doe...",09/08/2021,24/07/2021,< 60
...,...,...,...,...,...,...,...
5944,66,M,Aracaju,"Diabetes mellitus, Hipertensão, Obesidade",06/04/2020,01/04/2020,60-69
5945,90,F,Aracaju,Hipertensão arterial,04/04/2020,04/04/2020,> 80
5946,61,F,Aracaju,"Diabetes mellitus, Hipertensão arterial, cardi...",02/04/2020,30/03/2020,60-69
5947,60,M,Aracaju,"Hipertensão Arterial, Tabagismo",02/04/2020,01/04/2020,60-69


In [5]:
data_process = data_obitos.drop(columns=['sexo','municipio','comorbidade','data_confirmacao'])
data_process['data_obito'] = pd.to_datetime(data_process.data_obito, format='%d/%m/%Y')
data_process['data_obito_m_y'] = data_process['data_obito'].dt.strftime('%Y-%m')
data_count = data_process[['data_obito_m_y','idade']].groupby(by=['data_obito_m_y']).count()
data_process = data_process[['data_obito_m_y','idade_faixa','idade']].groupby(by=['data_obito_m_y','idade_faixa']).count()
for (index, rows) in data_process.iterrows():
    data_process.at[index, 'percent'] = float(rows['idade'] / data_count.loc[index[0]].idade)
data_process.reset_index(inplace=True)
data_process.rename(columns={'idade': 'total'}, inplace=True)
data_process

,data_obito_m_y,idade_faixa,total,percent
0,2020-04,60-69,4,0.307692
1,2020-04,70-69,2,0.153846
2,2020-04,< 60,3,0.230769
3,2020-04,> 80,4,0.307692
4,2020-05,60-69,49,0.234450
...,...,...,...,...
63,2021-07,> 80,29,0.181250
64,2021-08,60-69,4,0.173913
65,2021-08,70-69,5,0.217391
66,2021-08,< 60,9,0.391304


In [6]:
#data_final = data_process.drop(columns=['total'])
data_final = data_process.pivot_table(values=['percent', 'total'], index=['data_obito_m_y'], columns='idade_faixa')
data_final.reset_index(inplace=True)
data_final.rename(columns={'data_obito_m_y': 'data', '60-69': 'faixa_6069', '70-69': 'faixa_7079', '< 60': 'faixa_menor_60', '> 80': 'faixa_maior_80', '> 60': 'faixa_maior_60'}, inplace=True)
data_final.columns = data_final.columns.get_level_values(0) + "_" + data_final.columns.get_level_values(1)
data_final.rename(columns={'data_': 'data'},inplace=True)
data_final

,data,percent_faixa_6069,percent_faixa_7079,percent_faixa_menor_60,percent_faixa_maior_80,total_faixa_6069,total_faixa_7079,total_faixa_menor_60,total_faixa_maior_80
0,2020-04,0.307692,0.153846,0.230769,0.307692,4,2,3,4
1,2020-05,0.234450,0.196172,0.373206,0.196172,49,41,78,41
2,2020-06,0.203947,0.233553,0.325658,0.236842,124,142,198,144
3,2020-07,0.221485,0.265252,0.278515,0.234748,167,200,210,177
4,2020-08,0.226300,0.247706,0.247706,0.278287,74,81,81,91
5,2020-09,0.190184,0.233129,0.306748,0.269939,31,38,50,44
6,2020-10,0.251572,0.245283,0.226415,0.276730,40,39,36,44
7,2020-11,0.266055,0.275229,0.201835,0.256881,29,30,22,28
8,2020-12,0.176000,0.244000,0.284000,0.296000,44,61,71,74
9,2021-01,0.227848,0.223629,0.345992,0.202532,54,53,82,48


In [7]:
data_final['increase_6069'] = data_final['total_faixa_6069'].pct_change()
data_final['increase_7079'] = data_final['total_faixa_7079'].pct_change()
data_final['increase_menor_60'] = data_final['total_faixa_menor_60'].pct_change()
data_final['increase_maior_80'] = data_final['total_faixa_maior_80'].pct_change()
data_final.fillna(0, inplace=True)
data_final

,data,percent_faixa_6069,percent_faixa_7079,percent_faixa_menor_60,percent_faixa_maior_80,total_faixa_6069,total_faixa_7079,total_faixa_menor_60,total_faixa_maior_80,increase_6069,increase_7079,increase_menor_60,increase_maior_80
0,2020-04,0.307692,0.153846,0.230769,0.307692,4,2,3,4,0.000000,0.000000,0.000000,0.000000
1,2020-05,0.234450,0.196172,0.373206,0.196172,49,41,78,41,11.250000,19.500000,25.000000,9.250000
2,2020-06,0.203947,0.233553,0.325658,0.236842,124,142,198,144,1.530612,2.463415,1.538462,2.512195
3,2020-07,0.221485,0.265252,0.278515,0.234748,167,200,210,177,0.346774,0.408451,0.060606,0.229167
4,2020-08,0.226300,0.247706,0.247706,0.278287,74,81,81,91,-0.556886,-0.595000,-0.614286,-0.485876
5,2020-09,0.190184,0.233129,0.306748,0.269939,31,38,50,44,-0.581081,-0.530864,-0.382716,-0.516484
6,2020-10,0.251572,0.245283,0.226415,0.276730,40,39,36,44,0.290323,0.026316,-0.280000,0.000000
7,2020-11,0.266055,0.275229,0.201835,0.256881,29,30,22,28,-0.275000,-0.230769,-0.388889,-0.363636
8,2020-12,0.176000,0.244000,0.284000,0.296000,44,61,71,74,0.517241,1.033333,2.227273,1.642857
9,2021-01,0.227848,0.223629,0.345992,0.202532,54,53,82,48,0.227273,-0.131148,0.154930,-0.351351


In [8]:
#data_final = data_final[:-1]
data_final

,data,percent_faixa_6069,percent_faixa_7079,percent_faixa_menor_60,percent_faixa_maior_80,total_faixa_6069,total_faixa_7079,total_faixa_menor_60,total_faixa_maior_80,increase_6069,increase_7079,increase_menor_60,increase_maior_80
0,2020-04,0.307692,0.153846,0.230769,0.307692,4,2,3,4,0.000000,0.000000,0.000000,0.000000
1,2020-05,0.234450,0.196172,0.373206,0.196172,49,41,78,41,11.250000,19.500000,25.000000,9.250000
2,2020-06,0.203947,0.233553,0.325658,0.236842,124,142,198,144,1.530612,2.463415,1.538462,2.512195
3,2020-07,0.221485,0.265252,0.278515,0.234748,167,200,210,177,0.346774,0.408451,0.060606,0.229167
4,2020-08,0.226300,0.247706,0.247706,0.278287,74,81,81,91,-0.556886,-0.595000,-0.614286,-0.485876
5,2020-09,0.190184,0.233129,0.306748,0.269939,31,38,50,44,-0.581081,-0.530864,-0.382716,-0.516484
6,2020-10,0.251572,0.245283,0.226415,0.276730,40,39,36,44,0.290323,0.026316,-0.280000,0.000000
7,2020-11,0.266055,0.275229,0.201835,0.256881,29,30,22,28,-0.275000,-0.230769,-0.388889,-0.363636
8,2020-12,0.176000,0.244000,0.284000,0.296000,44,61,71,74,0.517241,1.033333,2.227273,1.642857
9,2021-01,0.227848,0.223629,0.345992,0.202532,54,53,82,48,0.227273,-0.131148,0.154930,-0.351351


In [9]:
data_final.to_csv("output/obitos_idade_mes.csv", index=False)

## UTI

In [10]:
from datetime import datetime
from calendar import monthrange

def last_day_of_month(date_value):
    return date_value.replace(day = monthrange(date_value.year, date_value.month)[1])

In [11]:
last_day_of_month(datetime.strptime('2020-02-15', '%Y-%m-%d'))

datetime.datetime(2020, 2, 29, 0, 0)

In [12]:
def first_day_of_month(date_value):
    return date_value.replace(day=1)


In [13]:
first_day_of_month(datetime.strptime('2020-01-31', '%Y-%m-%d'))

datetime.datetime(2020, 1, 1, 0, 0)

In [14]:
map_columns = {
    'Data': 'data',
    'Leitos Pública': 'leitos_publico',
    'Ocupação pública': 'ocupacao_publico',
    'Leitos Privada': 'leitos_privado',
    'Ocupação Privada': 'ocupacao_privado',
    '% Pública': 'taxa_publico',
    '% Privada': 'taxa_privado'
}

dados_uti = pd.read_excel("data_general/ocupacao_uti.xlsx")
dados_uti.rename(columns=map_columns, inplace=True)
dados_uti = dados_uti.iloc[1:]
dados_uti.reset_index(inplace=True, drop=True)
dados_uti

,data,leitos_publico,ocupacao_publico,leitos_privado,ocupacao_privado,taxa_publico,taxa_privado
0,2021-01-01,189,117,67,48,0.619048,0.716418
1,2021-01-02,189,120,68,46,0.634921,0.676471
2,2021-01-03,189,122,68,50,0.645503,0.735294
3,2021-01-04,195,130,68,43,0.666667,0.632353
4,2021-01-05,195,126,74,43,0.646154,0.581081
...,...,...,...,...,...,...,...
224,2021-08-13,137,44,54,10,0.321168,0.185185
225,2021-08-14,137,41,54,10,0.299270,0.185185
226,2021-08-15,137,36,54,10,0.262774,0.185185
227,2021-08-16,137,41,54,10,0.299270,0.185185


In [15]:
dados_uti.to_csv("output/historico_uti.csv", index=False)

In [16]:
dados_uti['last_date'] = dados_uti['data'].apply(first_day_of_month)
dados_uti['compare'] = dados_uti['data'] == dados_uti['last_date']
dados_uti.loc[list(dados_uti.index)[-1], 'compare'] = True
dados_uti

,data,leitos_publico,ocupacao_publico,leitos_privado,ocupacao_privado,taxa_publico,taxa_privado,last_date,compare
0,2021-01-01,189,117,67,48,0.619048,0.716418,2021-01-01,True
1,2021-01-02,189,120,68,46,0.634921,0.676471,2021-01-01,False
2,2021-01-03,189,122,68,50,0.645503,0.735294,2021-01-01,False
3,2021-01-04,195,130,68,43,0.666667,0.632353,2021-01-01,False
4,2021-01-05,195,126,74,43,0.646154,0.581081,2021-01-01,False
...,...,...,...,...,...,...,...,...,...
224,2021-08-13,137,44,54,10,0.321168,0.185185,2021-08-01,False
225,2021-08-14,137,41,54,10,0.299270,0.185185,2021-08-01,False
226,2021-08-15,137,36,54,10,0.262774,0.185185,2021-08-01,False
227,2021-08-16,137,41,54,10,0.299270,0.185185,2021-08-01,False


In [17]:
dados_uti_month = dados_uti[dados_uti['compare'] == True]

In [18]:
dados_uti_month = dados_uti_month.drop(columns=['leitos_publico', 'leitos_privado','taxa_publico','taxa_privado','last_date','compare'])
dados_uti_month

,data,ocupacao_publico,ocupacao_privado
0,2021-01-01,117,48
31,2021-02-01,90,61
59,2021-03-01,98,107
90,2021-04-01,206,183
120,2021-05-01,220,190
151,2021-06-01,227,165
181,2021-07-01,181,109
212,2021-08-01,49,26
228,2021-08-17,43,10


In [19]:
dados_uti_month['increase_publico'] = dados_uti_month['ocupacao_publico'].pct_change()
dados_uti_month['increase_privado'] = dados_uti_month['ocupacao_privado'].pct_change()
dados_uti_month.fillna(0, inplace=True)
dados_uti_month

,data,ocupacao_publico,ocupacao_privado,increase_publico,increase_privado
0,2021-01-01,117,48,0.000000,0.000000
31,2021-02-01,90,61,-0.230769,0.270833
59,2021-03-01,98,107,0.088889,0.754098
90,2021-04-01,206,183,1.102041,0.710280
120,2021-05-01,220,190,0.067961,0.038251
151,2021-06-01,227,165,0.031818,-0.131579
181,2021-07-01,181,109,-0.202643,-0.339394
212,2021-08-01,49,26,-0.729282,-0.761468
228,2021-08-17,43,10,-0.122449,-0.615385


In [20]:
#dados_uti_month = dados_uti_month[:-1]
dados_uti_month

,data,ocupacao_publico,ocupacao_privado,increase_publico,increase_privado
0,2021-01-01,117,48,0.000000,0.000000
31,2021-02-01,90,61,-0.230769,0.270833
59,2021-03-01,98,107,0.088889,0.754098
90,2021-04-01,206,183,1.102041,0.710280
120,2021-05-01,220,190,0.067961,0.038251
151,2021-06-01,227,165,0.031818,-0.131579
181,2021-07-01,181,109,-0.202643,-0.339394
212,2021-08-01,49,26,-0.729282,-0.761468
228,2021-08-17,43,10,-0.122449,-0.615385


In [21]:
dados_uti_month.to_csv("output/historico_uti_mes.csv", index=False)

In [22]:
dados_uti_hospitais = pd.read_excel("data_general/ocupacao_uti_hospitais.xlsx")

In [23]:
dados_uti_hospitais

,Hospital,Leitos,Ocupação,Tx Ocupação,Rede
0,HUSE,11,9,0.818182,Público
1,Hospital Cirurgia – SUS,20,8,0.400000,Público
2,Hospital Universitário Lagarto,20,1,0.050000,Público
3,Hospital da Policia Militar,18,6,0.333333,Público
4,Hospital Regional – Estância,18,5,0.277778,Público
5,Hospital N. Sra da Conceição,20,1,0.050000,Público
6,Maternidade N. Sra de Lourdes (UTIN),6,1,0.166667,Público
7,Hospital São José,6,2,0.333333,Público
8,Hospital Santa Izabel - SUS,8,6,0.750000,Público
9,HU - Aracaju,10,4,0.400000,Público


In [24]:
dados_uti_hospitais.rename(columns={
    'Hospital': 'nome_hospital',
    'Leitos': 'total_leitos',
    'Ocupação': 'total_ocupacao',
    'Tx Ocupação': 'taxa_ocupacao',
    'Rede': 'tipo_rede'
})

,nome_hospital,total_leitos,total_ocupacao,taxa_ocupacao,tipo_rede
0,HUSE,11,9,0.818182,Público
1,Hospital Cirurgia – SUS,20,8,0.400000,Público
2,Hospital Universitário Lagarto,20,1,0.050000,Público
3,Hospital da Policia Militar,18,6,0.333333,Público
4,Hospital Regional – Estância,18,5,0.277778,Público
5,Hospital N. Sra da Conceição,20,1,0.050000,Público
6,Maternidade N. Sra de Lourdes (UTIN),6,1,0.166667,Público
7,Hospital São José,6,2,0.333333,Público
8,Hospital Santa Izabel - SUS,8,6,0.750000,Público
9,HU - Aracaju,10,4,0.400000,Público
